In [3]:
import os

In [4]:
%pwd

'e:\\projects\\Delivery-time-prediction-for-food-devlivery-industry\\research'

In [5]:
os.chdir("../")

In [39]:
%pwd

'e:\\projects\\Delivery-time-prediction-for-food-devlivery-industry'

In [40]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataCleaningConfig:
  root_dir: Path
  data_input_dir: Path
  preprocess_data_dir: Path

In [41]:
from pathlib import Path

CONFIG_FILE_PATH = Path("E:\projects\Delivery-time-prediction-for-food-devlivery-industry\config\config.yaml")
PARAMS_FILE_PATH = Path("E:\projects\Delivery-time-prediction-for-food-devlivery-industry\params.yaml")
SCHEMA_FILE_PATH = Path("E:\projects\Delivery-time-prediction-for-food-devlivery-industry\schema.yaml")

In [42]:
print(CONFIG_FILE_PATH)

E:\projects\Delivery-time-prediction-for-food-devlivery-industry\config\config.yaml


In [43]:
from Deliveryprediction.constants import *
from Deliveryprediction.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_cleaning_config(self) -> DataCleaningConfig:
        config = self.config.data_cleaning

        create_directories([config.root_dir])

        data_cleaning_config = DataCleaningConfig(
            root_dir = config.root_dir,
            data_input_dir = config.data_input_dir,
            preprocess_data_dir = config.preprocess_data_dir
           )

        return data_cleaning_config

In [44]:
import numpy as np
import pandas as pd
from pathlib import Path

In [45]:
class DataCleaning:
    def __init__(self, config:  DataCleaningConfig):
         self.config = config

    def load_data(self) -> pd.DataFrame:
        data = pd.read_csv(self.config.data_input_dir)
        return data
    
    columns_to_drop =  ['rider_id',
                    'restaurant_latitude',
                    'restaurant_longitude',
                    'delivery_latitude',
                    'delivery_longitude',
                    'order_date',
                    "order_time_hour",
                    "order_day",
                    "city_name",
                    "order_day_of_week",
                    "order_month"]
    
    def change_column_names(self,data: pd.DataFrame) -> pd.DataFrame:
        return (
        data.rename(str.lower,axis=1)
        .rename({
            "delivery_person_id" : "rider_id",
            "delivery_person_age": "age",
            "delivery_person_ratings": "ratings",
            "delivery_location_latitude": "delivery_latitude",
            "delivery_location_longitude": "delivery_longitude",
            "time_orderd": "order_time",
            "time_order_picked": "order_picked_time",
            "weatherconditions": "weather",
            "road_traffic_density": "traffic",
            "city": "city_type",
            "time_taken(min)": "time_taken"},axis=1)
    )

    def data_cleaning(self,data: pd.DataFrame) -> pd.DataFrame:
        minors_data = data.loc[data['age'].astype('float') < 18]
        minor_index = minors_data.index.tolist()
        six_star_data = data.loc[data['ratings'] == "6"]
        six_star_index = six_star_data.index.tolist()
        return (
            data
            .drop(columns="id")
            .drop(index=minor_index)                                                # Minor riders in data dropped
            .drop(index=six_star_index)                                             # six star rated drivers dropped
            .replace("NaN ",np.nan)                                                 # missing values in the data
            .assign(
                # city column out of rider id
                city_name = lambda x: x['rider_id'].str.split("RES").str.get(0),
                # convert age to float
                age = lambda x: x['age'].astype(float),
                # convert ratings to float
                ratings = lambda x: x['ratings'].astype(float),
                # absolute values for location based columns
                restaurant_latitude = lambda x: x['restaurant_latitude'].abs(),
                restaurant_longitude = lambda x: x['restaurant_longitude'].abs(),
                delivery_latitude = lambda x: x['delivery_latitude'].abs(),
                delivery_longitude = lambda x: x['delivery_longitude'].abs(),
                # order date to datetime and feature extraction
                order_date = lambda x: pd.to_datetime(x['order_date'],
                                                    dayfirst=True),
                order_day = lambda x: x['order_date'].dt.day,
                order_month = lambda x: x['order_date'].dt.month,
                order_day_of_week = lambda x: x['order_date'].dt.day_name().str.lower(),
                is_weekend = lambda x: (x['order_date']
                                        .dt.day_name()
                                        .isin(["Saturday","Sunday"])
                                        .astype(int)),
                # time based columns
                order_time = lambda x: pd.to_datetime(x['order_time'],
                                                    format='mixed'),
                order_picked_time = lambda x: pd.to_datetime(x['order_picked_time'],
                                                            format='mixed'),
                # time taken to pick order
                pickup_time_minutes = lambda x: (
                                                (x['order_picked_time'] - x['order_time'])
                                                .dt.seconds / 60
                                                ),
                # hour in which order was placed
                order_time_hour = lambda x: x['order_time'].dt.hour,
                # time of the day when order was placed
                order_time_of_day = lambda x: (
                                    x['order_time_hour'].pipe(self.time_of_day)),
                # categorical columns
                weather = lambda x: (
                                    x['weather']
                                    .str.replace("conditions ","")
                                    .str.lower()
                                    .replace("nan",np.nan)),
                traffic = lambda x: x["traffic"].str.rstrip().str.lower(),
                type_of_order = lambda x: x['type_of_order'].str.rstrip().str.lower(),
                type_of_vehicle = lambda x: x['type_of_vehicle'].str.rstrip().str.lower(),
                festival = lambda x: x['festival'].str.rstrip().str.lower(),
                city_type = lambda x: x['city_type'].str.rstrip().str.lower(),
                # multiple deliveries column
                multiple_deliveries = lambda x: x['multiple_deliveries'].astype(float),
                # target column modifications
                time_taken = lambda x: (x['time_taken']
                                        .str.replace("(min) ","")
                                        .astype(int)))
            .drop(columns=["order_time","order_picked_time"])
        )
    
    def clean_lat_long(self,data: pd.DataFrame, threshold: float=1.0) -> pd.DataFrame:
        location_columns = ['restaurant_latitude',
                            'restaurant_longitude',
                            'delivery_latitude',
                            'delivery_longitude']

        return (
            data
            .assign(**{
                col: (
                    np.where(data[col] < threshold, np.nan, data[col].values)
                )
                for col in location_columns
            })
        )

    
    # extract day, day name, month and year
    def extract_datetime_features(self,ser: pd.Series) -> pd.DataFrame:
        date_col = pd.to_datetime(ser,dayfirst=True)

        return (
            pd.DataFrame(
                {
                    "day": date_col.dt.day,
                    "month": date_col.dt.month,
                    "year": date_col.dt.year,
                    "day_of_week": date_col.dt.day_name(),
                    "is_weekend": date_col.dt.day_name().isin(["Saturday","Sunday"]).astype(int)
                }
            ))
            

        
    def time_of_day(self, ser: pd.Series):

        return(
            pd.cut(ser,bins=[0,6,12,17,20,24],right=True,
                labels=["after_midnight","morning","afternoon","evening","night"])
        )



    def calculate_haversine_distance(self,data: pd.DataFrame) -> pd.DataFrame:
        location_columns = ['restaurant_latitude',
                            'restaurant_longitude',
                            'delivery_latitude',
                            'delivery_longitude']
        
        lat1 = data[location_columns[0]]
        lon1 = data[location_columns[1]]
        lat2 = data[location_columns[2]]
        lon2 = data[location_columns[3]]

        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(
            dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2

        c = 2 * np.arcsin(np.sqrt(a))
        distance = 6371 * c

        return ( data.assign(
                distance = distance))


    def create_distance_type(self,data: pd.DataFrame) -> pd.DataFrame:
        return(
            data
            .assign(
                    distance_type = pd.cut(data["distance"],bins=[0,5,10,15,25],
                                            right=False,labels=["short","medium","long","very_long"])
        ))



    def drop_columns(self,data: pd.DataFrame, columns: list) -> pd.DataFrame:
        data=data.drop(columns=columns)
        #cleaned_data= data.dropna()
        return data
    
    def save_data(self,data: pd.DataFrame):
            data = data.dropna()
            #df.to_csv(file_path_pandas, index=False)
            return data.to_csv(self.config.preprocess_data_dir,index=False)
    
    def perform_data_cleaning(self, data: pd.DataFrame):
    
        cleaned_data = (
            data
            .pipe(self.change_column_names)
            .pipe(self.data_cleaning)
            .pipe(self.clean_lat_long)
            .pipe(self.calculate_haversine_distance)
            .pipe(self.create_distance_type)
            .pipe(self.drop_columns,columns=self.columns_to_drop)
            .pipe(self.save_data)
        )
    
        return cleaned_data

In [53]:
config = ConfigurationManager()
data_cleaning_config = config.get_data_cleaning_config()
cleaning_data = DataCleaning(config=data_cleaning_config)
data=cleaning_data.load_data()
cleaning_data.perform_data_cleaning(data)

[2025-02-10 15:23:21,649: INFO: common: yaml file: E:\projects\Delivery-time-prediction-for-food-devlivery-industry\config\config.yaml loaded successfully]
[2025-02-10 15:23:21,652: INFO: common: yaml file: E:\projects\Delivery-time-prediction-for-food-devlivery-industry\params.yaml loaded successfully]
[2025-02-10 15:23:21,659: INFO: common: yaml file: E:\projects\Delivery-time-prediction-for-food-devlivery-industry\schema.yaml loaded successfully]
[2025-02-10 15:23:21,663: INFO: common: created directory at: artifacts]
[2025-02-10 15:23:21,666: INFO: common: created directory at: artifacts/data_cleaning]


In [ ]:
# def clean_lat_long(data: pd.DataFrame, threshold: float=1.0) -> pd.DataFrame:
#         location_columns = ['restaurant_latitude',
#                             'restaurant_longitude',
#                             'delivery_latitude',
#                             'delivery_longitude']

#         return (
#             data
#             .assign(**{
#                 col: (
#                     np.where(data[col] < threshold, np.nan, data[col].values)
#                 )
#                 for col in location_columns
#             })
#         )

    
# # extract day, day name, month and year
# def extract_datetime_features(ser: pd.Series) -> pd.DataFrame:
#     date_col = pd.to_datetime(ser,dayfirst=True)

#     return (
#         pd.DataFrame(
#             {
#                 "day": date_col.dt.day,
#                 "month": date_col.dt.month,
#                 "year": date_col.dt.year,
#                 "day_of_week": date_col.dt.day_name(),
#                 "is_weekend": date_col.dt.day_name().isin(["Saturday","Sunday"]).astype(int)
#             }
#         ))
        

    
# def time_of_day(ser: pd.Series):

#     return(
#         pd.cut(ser,bins=[0,6,12,17,20,24],right=True,
#             labels=["after_midnight","morning","afternoon","evening","night"])
#     )



# def calculate_haversine_distance(data: pd.DataFrame) -> pd.DataFrame:
#     location_columns = ['restaurant_latitude',
#                         'restaurant_longitude',
#                         'delivery_latitude',
#                         'delivery_longitude']
    
#     lat1 = data[location_columns[0]]
#     lon1 = data[location_columns[1]]
#     lat2 = data[location_columns[2]]
#     lon2 = data[location_columns[3]]

#     lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

#     dlon = lon2 - lon1
#     dlat = lat2 - lat1

#     a = np.sin(
#         dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2

#     c = 2 * np.arcsin(np.sqrt(a))
#     distance = 6371 * c

#     return ( data.assign(
#             distance = distance))


# def create_distance_type(data: pd.DataFrame) -> pd.DataFrame:
#     return(
#         data
#         .assign(
#                 distance_type = pd.cut(data["distance"],bins=[0,5,10,15,25],
#                                         right=False,labels=["short","medium","long","very_long"])
#     ))



# def drop_columns(self,data: pd.DataFrame, columns: list) -> pd.DataFrame:
#     data=data.drop(columns=columns)
#     cleaned_data= self.data.dropna()
#     return cleaned_data

# def save_data(data: pd.DataFrame):
        # return data.to_csv(self.config.preprocess_data_dir,index=False)

In [ ]:
# data = pd.read_csv("E://projects//Delivery-time-prediction-for-food-devlivery-industry//artifacts//data_ingestion//swiggy.csv")

          

# columns_to_drop =  ['rider_id',
#             'restaurant_latitude',
#             'restaurant_longitude',
#             'delivery_latitude',
#             'delivery_longitude',
#             'order_date',
#             "order_time_hour",
#             "order_day",
#             "city_name",
#             "order_day_of_week",
#             "order_month"]


# def change_column_names(data: pd.DataFrame) -> pd.DataFrame:
#     return (
#     data.rename(str.lower,axis=1)
#     .rename({
#         "delivery_person_id" : "rider_id",
#         "delivery_person_age": "age",
#         "delivery_person_ratings": "ratings",
#         "delivery_location_latitude": "delivery_latitude",
#         "delivery_location_longitude": "delivery_longitude",
#         "time_orderd": "order_time",
#         "time_order_picked": "order_picked_time",
#         "weatherconditions": "weather",
#         "road_traffic_density": "traffic",
#         "city": "city_type",
#         "time_taken(min)": "time_taken"},axis=1)
#     )

# data = change_column_names(data)

# def data_cleaning(data: pd.DataFrame) -> pd.DataFrame:
#     minors_data = data.loc[data['age'].astype('float') < 18]
#     minor_index = minors_data.index.tolist()
#     six_star_data = data.loc[data['ratings'] == "6"]
#     six_star_index = six_star_data.index.tolist()

#     return (
#         data
#         .drop(columns="id")
#         .drop(index=minor_index)                                                # Minor riders in data dropped
#         .drop(index=six_star_index)                                             # six star rated drivers dropped
#         .replace("NaN ",np.nan)                                                 # missing values in the data
#         .assign(
#             # city column out of rider id
#             city_name = lambda x: x['rider_id'].str.split("RES").str.get(0),
#             # convert age to float
#             age = lambda x: x['age'].astype(float),
#             # convert ratings to float
#             ratings = lambda x: x['ratings'].astype(float),
#             # absolute values for location based columns
#             restaurant_latitude = lambda x: x['restaurant_latitude'].abs(),
#             restaurant_longitude = lambda x: x['restaurant_longitude'].abs(),
#             delivery_latitude = lambda x: x['delivery_latitude'].abs(),
#             delivery_longitude = lambda x: x['delivery_longitude'].abs(),
#             # order date to datetime and feature extraction
#             order_date = lambda x: pd.to_datetime(x['order_date'],
#                                                 dayfirst=True),
#             order_day = lambda x: x['order_date'].dt.day,
#             order_month = lambda x: x['order_date'].dt.month,
#             order_day_of_week = lambda x: x['order_date'].dt.day_name().str.lower(),
#             is_weekend = lambda x: (x['order_date']
#                                     .dt.day_name()
#                                     .isin(["Saturday","Sunday"])
#                                     .astype(int)),
#             # time based columns
#             order_time = lambda x: pd.to_datetime(x['order_time'],
#                                                 format='mixed'),
#             order_picked_time = lambda x: pd.to_datetime(x['order_picked_time'],
#                                                         format='mixed'),
#             # time taken to pick order
#             pickup_time_minutes = lambda x: (
#                                             (x['order_picked_time'] - x['order_time'])
#                                             .dt.seconds / 60
#                                             ),
#             # hour in which order was placed
#             order_time_hour = lambda x: x['order_time'].dt.hour,
#             # time of the day when order was placed
#             order_time_of_day = lambda x: (
#                                 x['order_time_hour'].pipe(time_of_day)),
#             # categorical columns
#             weather = lambda x: (
#                                 x['weather']
#                                 .str.replace("conditions ","")
#                                 .str.lower()
#                                 .replace("nan",np.nan)),
#             traffic = lambda x: x["traffic"].str.rstrip().str.lower(),
#             type_of_order = lambda x: x['type_of_order'].str.rstrip().str.lower(),
#             type_of_vehicle = lambda x: x['type_of_vehicle'].str.rstrip().str.lower(),
#             festival = lambda x: x['festival'].str.rstrip().str.lower(),
#             city_type = lambda x: x['city_type'].str.rstrip().str.lower(),
#             # multiple deliveries column
#             multiple_deliveries = lambda x: x['multiple_deliveries'].astype(float),
#             # target column modifications
#             time_taken = lambda x: (x['time_taken']
#                                     .str.replace("(min) ","")
#                                     .astype(int)))
#         .drop(columns=["order_time","order_picked_time"])
#     )
    

In [73]:
data = data_cleaning(data)

In [74]:
data.head()

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,city_type,time_taken,city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,urban,24,INDO,19,3,saturday,1,15.0,11.0,morning
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,metropolitian,33,BANG,25,3,friday,0,5.0,19.0,evening
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,urban,26,BANG,19,3,saturday,1,15.0,8.0,morning
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,metropolitian,21,COIMB,5,4,tuesday,0,10.0,18.0,evening
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,metropolitian,30,CHEN,26,3,saturday,1,15.0,13.0,afternoon


In [47]:
def time_of_day(self,ser: pd.Series):

    return(
        pd.cut(ser,bins=[0,6,12,17,20,24],right=True,
            labels=["after_midnight","morning","afternoon","evening","night"])
    )

In [61]:
data.head()


,id,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,order_time,order_picked_time,weather,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,time_taken
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [66]:

minors_data = data.loc[data['age'].astype('float') < 18]
minor_index = minors_data.index.tolist()
six_star_data = data.loc[data['ratings'] == "6"]
six_star_index = six_star_data.index.tolist()

data.drop(columns="id")
data.drop(index=minor_index)                                                # Minor riders in data dropped
data.drop(index=six_star_index)                                             # six star rated drivers dropped
data.replace("NaN ",np.nan)                                                 # missing values in the data

# city column out of rid
city_name = lambda x: x['rider_id'].str.split("RES").str.get(0),
# convert age to float
age = lambda x: x['age'].astype(float),
# convert ratings to float
ratings = lambda x: x['ratings'].astype(float),
# absolute values for location based columns
restaurant_latitude = lambda x: x['restaurant_latitude'].abs(),
restaurant_longitude = lambda x: x['restaurant_longitude'].abs(),
delivery_latitude = lambda x: x['delivery_latitude'].abs(),
delivery_longitude = lambda x: x['delivery_longitude'].abs(),
# order date to datetime and feature extraction
order_date = lambda x: pd.to_datetime(x['order_date'],
                                    dayfirst=True),
order_day = lambda x: x['order_date'].dt.day,
order_month = lambda x: x['order_date'].dt.month,
order_day_of_week = lambda x: x['order_date'].dt.day_name().str.lower(),
is_weekend = lambda x: (x['order_date']
                        .dt.day_name()
                        .isin(["Saturday","Sunday"])
                        .astype(int)),
# time based columns
order_time = lambda x: pd.to_datetime(x['order_time'],
                                    format='mixed'),
order_picked_time = lambda x: pd.to_datetime(x['order_picked_time'],
                                            format='mixed'),
# time taken to pick order
pickup_time_minutes = lambda x: (
                                (x['order_picked_time'] - x['order_time'])
                                .dt.seconds / 60
                                ),
# hour in which order was placed
order_time_hour = lambda x: x['order_time'].dt.hour,
# time of the day when order was placed
order_time_of_day = lambda x: (x['order_time_hour'].pipe(time_of_day))
weather = lambda x: (
                    x['weather']
                    .str.replace("conditions ","")
                    .str.lower()
                    .replace("nan",np.nan)),
traffic = lambda x: x["traffic"].str.rstrip().str.lower(),
type_of_order = lambda x: x['type_of_order'].str.rstrip().str.lower(),
type_of_vehicle = lambda x: x['type_of_vehicle'].str.rstrip().str.lower(),
festival = lambda x: x['festival'].str.rstrip().str.lower(),
city_type = lambda x: x['city_type'].str.rstrip().str.lower(),
# multiple deliveries column
multiple_deliveries = lambda x: x['multiple_deliveries'].astype(float),
# target column modifications
time_taken = lambda x: (x['time_taken']
                        .str.replace("(min) ","")
                        .astype(int))
#drop(columns=["order_time","order_picked_time"])

In [67]:
data.head()

,id,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,order_time,order_picked_time,weather,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,time_taken
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30
